In [ ]:
import notebook_setup
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from gpdm import GPDM
from gpdm_filter import GPDM_Filter
from gpdm_ukf import GPDM_UKF
from gpdm_pf import GPDM_PF

from dataset_utils.mocap_labels import RUN_TRIALS_TEST, WALK_TRIALS_TEST
import dataset_utils.select_joints as select_joints

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.30.7, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


Number of DOFs: 35


In [3]:
d = 3 # latent space dimension
DOFs = len(reduced_columns)  # number of degrees of freedom

dyn_back_step = 1 # Number of time steps to look back in the dynamics GP

# Initial values for hyperparameters
y_lambdas_init = np.ones(DOFs)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(dyn_back_step*d)  # Lengthscales for latent dynamics GP
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(dyn_back_step*d + 1)  # Linear coefficients for latent dynamics GP

In [4]:
model_dir = Path().cwd() / 'models' / 'epochs_1k'
walk_config = model_dir / 'walk_gpdm_config.pth'
walk_state = model_dir / 'walk_gpdm_state.pth'

walk_gpdm =  GPDM(
        D=DOFs,
        d=d,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )
walk_gpdm.load(walk_config, walk_state, False)
#walk_gpdm_filter = GPDM_UKF(walk_gpdm)
#walk_gpdm_filter = GPDM_Filter(walk_gpdm)
walk_gpdm_filter = GPDM_PF(walk_gpdm, num_particles=100)




GPDM correctly loaded


In [5]:
run_config = model_dir / 'run_gpdm_config.pth'
run_state = model_dir / 'run_gpdm_state.pth'

run_gpdm =  GPDM(
        D=DOFs,
        d=d,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )

run_gpdm.load(run_config, run_state, False)
#run_gpdm_filter = GPDM_UKF(run_gpdm)
#run_gpdm_filter = GPDM_Filter(run_gpdm)
run_gpdm_filter = GPDM_PF(run_gpdm, num_particles=100)




GPDM correctly loaded


In [ ]:
# Iterate over the test trials and classify them

for i, trial in enumerate(TEST_TRIALS):
    label = TRUTH_LABELS[i]

    # Get the data in an array format
    data_arr = trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::5, :]

    walk_gpdm_filter.reset()
    run_gpdm_filter.reset()

    walk_latents = []
    run_latents = []

    # Run each data point through the GPDM filter
    for timestep in range(data_arr.shape[0]):
        data_point = data_arr[timestep, :]
        walk_ll = walk_gpdm_filter.update(data_point)
        if walk_ll is None:
            walk_ll = walk_gpdm_filter.log_likelihood()
            walk_ig = walk_gpdm_filter.expected_information_gain()
            walk_latents.append(walk_gpdm_filter.mu)
        run_ll = run_gpdm_filter.update(data_point)
        if run_ll is None:
            run_ll = run_gpdm_filter.log_likelihood()
            run_ig = run_gpdm_filter.expected_information_gain()
            run_latents.append(run_gpdm_filter.mu)


    # Classify the trial using the final log likelihoods
    if walk_ll > run_ll:
        predicted_label = 0
    else:
        predicted_label = 1

    print(f"Trial {i}: True label: {label}, Predicted label: {predicted_label}, Walk LL: {walk_ll}, Run LL: {run_ll}")
    print(f"Walk IG: {walk_ig}, Run IG: {run_ig}")
    print(f"Walk residual: {np.linalg.norm(walk_gpdm_filter.residual)}, Run residual: {np.linalg.norm(run_gpdm_filter.residual)}")

    # Plot the latent space
    walk_latents = np.array(walk_latents)
    run_latents = np.array(run_latents)

    # threed latent space
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(walk_latents[:, 0], walk_latents[:, 1], walk_latents[:, 2], c='r', marker='o', label='walk')

    ax.scatter(run_latents[:, 0], run_latents[:, 1], run_latents[:, 2], c='b', marker='o', label='run')
    plt.legend()
    plt.show()

AttributeError: 'GPDM_PF' object has no attribute 'mu'